In [ ]:
# Transformers installation
! pip install transformers datasets evaluate accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00


# Text classification

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load emotions dataset

Start by loading the IMDb dataset from the 🤗 Datasets library:

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('/content/dataFinal.csv')
dataset = Dataset.from_pandas(df)

# Specify the percentage for the training set
train_percentage = 0.8

# Use the 'train_test_split' method to split the dataset
splits = dataset.train_test_split(test_size=1 - train_percentage, shuffle=True, seed=42)
train_data, test_data = splits['train'], splits['test']

# Create a DatasetDict containing 'train' and 'test' splits
data = DatasetDict({
    'train': train_data,
    'test': test_data
})

Then take a look at an example:

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10224
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2556
    })
})

In [ ]:
data["test"][20]

{'text': 'Je me sens inquiet en ouvrant la porte bleue', 'label': 4}

## Preprocess

The next step is to load a CamemBERT tokenizer to preprocess the `text` field:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("camembert-base")

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Create a preprocessing function to tokenize `text` and truncate sequences to be no longer than camemBERT's maximum input length:

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up `map` by setting `batched=True` to process multiple elements of the dataset at once:

In [ ]:
tokenized_emotions = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/10224 [00:00<?, ? examples/s]

Map:   0%|          | 0/2556 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the accuracy:

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [ ]:
id2label = {0: "tristesse", 1: "joie", 2: "degout", 3:"colere", 4:"peur", 5:"surpris"}
label2id = {"tristesse": 0, "joie":1,  "degout":2, "colere":3, "peur":4, "surpris":5}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "camembert-base", num_labels=6, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="translateddata_emotion_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_emotions["train"],
    eval_dataset=tokenized_emotions["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.177000,0.645832,0.814163
2,0.608800,0.417184,0.872848
3,0.406400,0.364712,0.893192


TrainOutput(global_step=1917, training_loss=0.6456454838696531, metrics={'train_runtime': 25511.3929, 'train_samples_per_second': 1.202, 'train_steps_per_second': 0.075, 'total_flos': 1392096588424128.0, 'train_loss': 0.6456454838696531, 'epoch': 3.0})

<Tip>

[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) applies dynamic padding by default when you pass `tokenizer` to it. In this case, you don't need to specify a data collator explicitly.

</Tip>

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/ac0hik/translateddata_emotion_classifier/tree/main/'

<Tip>

For a more in-depth example of how to finetune a model for text classification, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification-tf.ipynb).

</Tip>

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [ ]:
text = "aufait, je suis resté impressionné par plus de quelques entreprises"

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for sentiment analysis with your model, and pass your text to it:

In [ ]:
from transformers import AutoTokenizer


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/translateddata_emotion_classifier")
inputs = tokenizer(text, return_tensors="pt")

Pass your inputs to the model and return the `logits`:

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/translateddata_emotion_classifier")
with torch.no_grad():
    logits = model(**inputs).logits

Get the class with the highest probability, and use the model's `id2label` mapping to convert it to a text label:

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'surpris'